In [2]:
#MSA
from Bio.Seq import Seq
import Bio.SeqIO as SeqIO
import random
import numpy as np

sick_patient_seq = SeqIO.read('HBS.fasta', 'fasta')
print(sick_patient_seq)
print(sick_patient_seq.seq)


# number of individuals in each generation 
NUM_SEQ = 10
POP_SIZE = 10
SEQ_LEN = 15

def generate_sequence(base, length=5, prob=0.5):
    pattern = ""
    a = ["A","C","G","T"]
    
    for i in range(0, length):
        r = random.randrange(0,4)
        p = random.random()
        #print(r)
        if p >= prob:
            pattern += base[i]
        else:
            pattern += a[r]
            
    return pattern 

def generate_set(base):
    seq_set = []
    
    for i in range(0, NUM_SEQ):
        seq_set.append(generate_sequence(base, SEQ_LEN))
       
    return seq_set

base = 'ACATTTGCTTCTGACACAACTGTGTTCACTAGCAACCTCAAACAGACACCATGG'
sequences = generate_set(base)

ID: Sick
Name: Sick
Description: Sick patient HBB
Number of features: 0
Seq('ACATTTGCTTCTGACACAACTGTGTTCACTAGCAACCTCAAACAGACACCATGG...CAA', SingleLetterAlphabet())
ACATTTGCTTCTGACACAACTGTGTTCACTAGCAACCTCAAACAGACACCATGGTGCATCTGACTCCTGTGGAGAAGTCTGCCGTTACTGCCCTGTGGGGCAAGGTGAACGTGGATGAAGTTGGTGGTGAGGCCCTGGGCAGGCTGCTGGTGGTCTACCCTTGGACCCAGAGGTTCTTTGAGTCCTTTGGGGATCTGTCCACTCCTGATGCTGTTATGGGCAACCCTAAGGTGAAGGCTCATGGCAAGAAAGTGCTCGGTGCCTTTAGTGATGGCCTGGCTCACCTGGACAACCTCAAGGGCACCTTTGCCACACTGAGTGAGCTGCACTGTGACAAGCTGCACGTGGATCCTGAGAACTTCAGGCTCCTGGGCAACGTGCTGGTCTGTGTGCTGGCCCATCACTTTGGCAAAGAATTCACCCCACCAGTGCAGGCTGCCTATCAGAAAGTGGTGGCTGGTGTGGCTAATGCCCTGGCCCACAAGTATCACTAAGCTCGCTTTCTTGCTGTCCAATTTCTATTAAAGGTTCCTTTGTTCCCTAAGTCCAACTACTAAACTGGGGGATATTATGAAGGGCCTTGAGCATCTGGATTCTGCCTAATAAAAAACATTTATTTTCATTGCAA


In [84]:
#Creation of initial population(movement of each sequence is a number between MOV_LEFT and MOV_RIGHT)
def initial_population(sequences, population_size, max_gap=3, mov_prob=0.1):
    population = []
    num_seq = len(sequences)
    sequence_movement = []
    sequences_movement = []
    #movement probability \
    for _ in range(population_size):
        for i in range(0, num_seq):
            sequence_movement = []
            for j in range(0, SEQ_LEN):
                random_mov = np.random.random()
                if (random_mov < mov_prob):
                    #np.random.randint(max_gap, size=(num_seq, SEQ_LEN))
                    seq_movement = np.random.randint(max_gap)
                else:
                    seq_movement = 0
                sequence_movement.append(seq_movement)
            sequences_movement.append(sequence_movement)
        population.append(sequences_movement)

    return population

#now a matrix of gaps for each character
initial_pop = initial_population(sequences, POP_SIZE)
#print(initial_pop)

In [98]:
def pad_sequence(pop, seqs, max_gap):
    new_seq = []
    new_sequences = []
    max_seq_len = max_gap + SEQ_LEN
    #print(max_seq_len)
    for p in pop:
        new_seq = []
        for seq, movement in zip(seqs,p):
            if movement == 0:
#                 new_seq += seq
                new_seq.append(seq)
            else:
                for i in range(0, movement):
#                     new_seq += '-'
                    new_seq.append('-')
                #new_seq += seq
                new_seq.append(seq)
        new_seq_length = len(new_seq)
        if new_seq_length < max_seq_len:
            diff = max_seq_len - new_seq_length
            for i in range(0, diff):
                new_seq.append('-')
        new_sequences.append(new_seq) 
    #print(new_sequences)
    return new_sequences

def split(word):
    return [char for char in word]

def padded_sequences(initial_pop):
    #padded based on population
    padded_sequences = []
    max_gap=0
    for i in range(0,len(initial_pop)):
        for j in range(0, len(initial_pop[i])):
            print(initial_pop[i][j])
            if sum(initial_pop[i][j]) > max_gap:
                
                max_gap = sum(initial_pop[i][j]) + SEQ_LEN
    print(max_gap)
    #now for the max gap pad all the sequences and store them
    for i in range(0, POP_SIZE):
        padded_sequence = pad_sequence(initial_pop[i], sequences[i], max_gap)
        padded_sequences.append(padded_sequence)
    return padded_sequences

padded_seqs = padded_sequences(initial_pop)
#print(padded_seqs)

[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [99]:
def calc_score(seqs):
    s = np.array(seqs, ndmin=2)   
    score = 0
    
    #looking for num of cols
    print(s.shape)
    for i in range(0,s.shape[1]):
        #print(len(set(s[:,i])))
        score += len(set(s[:,i]))
    return score

def MSA(ps, top=5):
    scores = []
    top_scores = []
    if top > NUM_SEQ:
        print("Top must be smaller or the same num as number of sequences.")
        return
    for i in range(0, len(ps)):
        scores.append((initial_pop[i],calc_score(ps[i])))
    scores = sorted(scores, key=lambda x: x[1])
    
    for i in range(0, top):
        top_scores.append(scores[i])

    return top_scores 

In [100]:
def mutate(chromosome, mutated = 1, p = 0.1):
    mut = random.random()
    new_chr = chromosome
    if(mut < p):
        for j in range(0, mutated):
            #picking random character position to be mutated
            pos = random.randrange(0, len(new_chr))
            r = random.randrange(0,3)
            new_chr[pos] = r
    return new_chr

def crossover_and_generate_new(top_chromosomes):
    global POP_SIZE
    new_pop = []
    #generating new population
    for i in range(0,round(POP_SIZE/2)):
        r = random.randrange(0, NUM_SEQ-1)
        r_ch1 = random.randrange(0, len(top_chromosomes))
        r_ch2 = random.randrange(0, len(top_chromosomes))
        
        #crossover
        new_chr = top_chromosomes[r_ch1][0][:r] + top_chromosomes[r_ch2][0][r:]
        new_chr2 = top_chromosomes[r_ch1][0][r:] + top_chromosomes[r_ch2][0][:r]
        
        #performing mutation if mut is bigger than p
        new_chr = mutate(new_chr)
        new_chr2 = mutate(new_chr2)
        
        new_pop.append(new_chr)
        new_pop.append(new_chr2)
        
    return new_pop

In [102]:
top_chromosomes = MSA(padded_seqs)
#print(top_chromosomes)

#iterations until some convergence
num_iter = 100
conv = []
for i in range(0, num_iter):
    #print(top_chromosomes)
    new_population = crossover_and_generate_new(top_chromosomes)
    #print(new_population)
    padded_seqs2 = padded_sequences(new_population)
    top_chromosomes2 = MSA(padded_seqs2)
    #print(top_chromosomes2)
    conv.append(top_chromosomes2)
    

(100, 32)
(100, 32)
(100, 32)
(100, 32)
(100, 32)
(100, 32)
(100, 32)
(100, 32)
(100, 32)
(100, 32)
[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]
[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0]
[0, 0, 0, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 1, 0]
[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
[2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0

TypeError: 'int' object is not iterable